# Regressions


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, models, transforms
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import util
import statsmodels.api as sm
from scipy import stats
import copy

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

In [2]:
# read df and images
size = 12000
df_ = pd.read_csv("data_process/df_merged_tract_large.csv")
df = df_.iloc[:size,]

In [3]:
# read latent vectors 
import pickle
with open('data_process/last_layer_dic_train.pickle', 'rb') as h:
    last_layer_dic_train = pickle.load(h)

with open('data_process/last_layer_dic_test.pickle', 'rb') as h:
    last_layer_dic_test = pickle.load(h)
    

In [4]:
# linear regression 
def Linear_eval(x_train_, x_test_, y_train_, y_test_):
    linear_mod = sm.OLS(y_train_, x_train_)
    linear_mod_res = linear_mod.fit()
    # eval
    train_mse = mean_squared_error(y_train_, linear_mod_res.predict(x_train_))
    test_mse = mean_squared_error(y_test_, linear_mod_res.predict(x_test_))
    train_r2 = r2_score(y_train_, linear_mod_res.predict(x_train_))
    test_r2 = r2_score(y_test_, linear_mod_res.predict(x_test_))
    return linear_mod_res, train_mse, test_mse, train_r2, test_r2

In [5]:
# linear regression with regularization
def Linear_reg_eval(x_train_, x_test_, y_train_, y_test_, method, alpha, L1_wt):
    linear_mod = sm.OLS(y_train_, x_train_)
    linear_mod_res = linear_mod.fit_regularized(method=method, alpha=alpha, L1_wt=L1_wt)
    # eval
    train_mse = mean_squared_error(y_train_, linear_mod_res.predict(x_train_))
    test_mse = mean_squared_error(y_test_, linear_mod_res.predict(x_test_))
    train_r2 = r2_score(y_train_, linear_mod_res.predict(x_train_))
    test_r2 = r2_score(y_test_, linear_mod_res.predict(x_test_))
    return linear_mod_res, train_mse, test_mse, train_r2, test_r2

In [6]:
def initialize_data_linear_reg(df, BE_var, output_var, input_var, last_layer_dic_train, last_layer_dic_test, size, input_structure):
    # output: x train and test, y train and test.
    y_ = df[output_var].values 
    y = copy.deepcopy(y_)
    x = df[input_var]
    BE = df[BE_var]
    
    # randomization. 
    shuffle_idx = np.arange(size)
    np.random.seed(0) # Keey this seed consistent across scripts.
    np.random.shuffle(shuffle_idx)
    train_ratio = 0.8 # Keey this consistent across scripts.

    # train test.
    y_train = y[shuffle_idx[:int(train_ratio*size)]].astype("float32")
    y_test = y[shuffle_idx[int(train_ratio*size):]].astype("float32")
    BE_train = BE.values[shuffle_idx[:int(train_ratio*size)]].astype("float32")
    BE_test = BE.values[shuffle_idx[int(train_ratio*size):]].astype("float32")
    x_train = x.values[shuffle_idx[:int(train_ratio*size)]].astype("float32")
    x_test = x.values[shuffle_idx[int(train_ratio*size):]].astype("float32")
    # 
    cnn_vector_train=last_layer_dic_train[output_var] 
    cnn_vector_test=last_layer_dic_test[output_var]
    
    if input_structure == 'BE linear':
        x_train_ = sm.add_constant(BE_train)
        x_test_ = sm.add_constant(BE_test)
        y_train_ = y_train[:]
        y_test_ = y_test[:]

    elif input_structure == 'BE quadratic':
        poly = PolynomialFeatures(2, interaction_only = False, include_bias=True)
        x_train_ = poly.fit_transform(BE_train)
        x_test_ = poly.fit_transform(BE_test)
        y_train_ = y_train[:]
        y_test_ = y_test[:]
        
    elif input_structure == 'NHTS linear':
        x_train_ = sm.add_constant(x_train)
        x_test_ = sm.add_constant(x_test)
        y_train_ = y_train[:]
        y_test_ = y_test[:]

    elif input_structure == 'NHTS quadratic': # I have concern about its computational problem.
        poly = PolynomialFeatures(2, interaction_only = False, include_bias=True)
        x_train_ = poly.fit_transform(x_train)
        x_test_ = poly.fit_transform(x_test)
        y_train_ = y_train[:]
        y_test_ = y_test[:]
        
    elif input_structure == 'BE and NHTS linear':
        x_train_ = sm.add_constant(np.concatenate([x_train, BE_train], axis = 1))
        x_test_ = sm.add_constant(np.concatenate([x_test, BE_test], axis = 1))
        y_train_ = y_train[:]
        y_test_ = y_test[:]
        
    elif input_structure == 'BE and NHTS quadratic': # I have concern about its computational problem. 
        poly = PolynomialFeatures(2, interaction_only = False, include_bias=True)
        x_train_ = poly.fit_transform(np.concatenate([x_train, BE_train], axis = 1))
        x_test_ = poly.fit_transform(np.concatenate([x_test, BE_test], axis = 1))
        y_train_ = y_train[:]
        y_test_ = y_test[:]

    elif input_structure == 'CNN and NHTS linear':
        x_train_ = sm.add_constant(np.concatenate([x_train, cnn_vector_train], axis = 1))
        x_test_ = sm.add_constant(np.concatenate([x_test, cnn_vector_test], axis = 1))
        y_train_ = y_train[:]
        y_test_ = y_test[:]

    elif input_structure == 'CNN BE NHTS linear':
        x_train_ = sm.add_constant(np.concatenate([x_train, BE_train, cnn_vector_train], axis = 1))
        x_test_ = sm.add_constant(np.concatenate([x_test, BE_test, cnn_vector_test], axis = 1))
        y_train_ = y_train[:]
        y_test_ = y_test[:]

    return x_train_, x_test_, y_train_, y_test_
    
# # test
# output_var = 'HHVEHCNT_mean'
# input_var = ['R_AGE_IMP_mean', 'HHSIZE_mean', 'HHFAMINC_mean', 'HBHTNRNT_mean', 'HBPPOPDN_mean', 'HBRESDN_mean', 
#       'R_SEX_IMP_2_mean', 'EDUC_2_mean', 'HH_RACE_2_mean', 'HOMEOWN_1_mean', 'HOMEOWN_2_mean',
#       'HBHUR_R_mean', 'HBHUR_S_mean', 'HBHUR_T_mean','HBHUR_U_mean']
# input_structure = 'BE and NHTS quadratic'
# x_train_, x_test_, y_train_, y_test_ = initialize_data_linear_reg(df, BE, output_var, input_var, size, input_structure)
# print(x_train_)
# print(y_train_)
# print(x_train_.shape)
# print(y_train_.shape)

In [7]:
# set up
output_var_list=['HHVEHCNT_mean_norm', 'HHVEHCNT_P_CAP_mean_norm', 'TRPTRANS_1_mean_norm', 'TRPTRANS_2_mean_norm', 'TRPTRANS_3_mean_norm']
# input_var=['R_AGE_IMP_mean', 
#            'HBHUR_R_mean', 'HBHUR_S_mean']
input_var=['R_AGE_IMP_mean', 'HHSIZE_mean', 'HHFAMINC_mean', 'HBHTNRNT_mean', 'HBPPOPDN_mean', 'HBRESDN_mean', 
           'R_SEX_IMP_2_mean', 'EDUC_2_mean', 'HH_RACE_2_mean', 'HOMEOWN_1_mean', 'HOMEOWN_2_mean',
           'HBHUR_R_mean', 'HBHUR_S_mean', 'HBHUR_T_mean','HBHUR_U_mean']
BE_var = ['density', 'diversity', 'design']
input_structure_list = ['BE linear', 'BE quadratic', 'NHTS linear', 'NHTS quadratic', 'BE and NHTS linear', 'BE and NHTS quadratic']


In [8]:
# one example regression.
output_var = "HHVEHCNT_mean_norm"
input_structure = 'BE linear'
x_train_, x_test_, y_train_, y_test_ = initialize_data_linear_reg(df, BE_var, output_var, input_var, last_layer_dic_train, last_layer_dic_test, size, input_structure)
linear_mod_res, train_mse, test_mse, train_r2, test_r2 = Linear_eval(x_train_, x_test_, y_train_, y_test_)
linear_mod_res.summary()
# Note: results show that 3Ds are very significant.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.078
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     270.3
Date:                Thu, 25 Jun 2020   Prob (F-statistic):          2.12e-168
Time:                        20:07:00   Log-Likelihood:                -13753.
No. Observations:                9600   AIC:                         2.751e+04
Df Residuals:                    9596   BIC:                         2.754e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3595      0.019     19.121      0.000       0.323       0.396
x1            -0.5548      0.077     -7.200      0.000      -0.706      -0.404
x2            -0.1004      0.033     -3.005      0.003      -0.166      -0.035
x3            -1.1654      0.086    -13.615      0.000      -1.333      -0.998
==============================================================================
Omnibus:                     3643.110   Durbin-Watson:                   2.020
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            26910.709
Skew:                           1.634   Prob(JB):                         0.00
Kurtosis:                      10.523   Cond. No.                         11.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
# iterate over models.
performance_handcrafted = {}

for output_var in output_var_list:
    print("-----")
    print(output_var)
    performance_handcrafted[output_var] = {}

    for input_structure in input_structure_list:
        print(input_structure)
        x_train_, x_test_, y_train_, y_test_ = initialize_data_linear_reg(df, BE_var, output_var, input_var, last_layer_dic_train, last_layer_dic_test, size, input_structure)
        linear_mod_res, train_mse, test_mse, train_r2, test_r2 = Linear_eval(x_train_, x_test_, y_train_, y_test_)
        # save models
        linear_mod_res.save("models/"+output_var+"_"+input_structure+".pickle")
        # save
        performance_handcrafted[output_var][input_structure] = {}
        performance_handcrafted[output_var][input_structure]['train_mse']=train_mse
        performance_handcrafted[output_var][input_structure]['test_mse']=test_mse
        performance_handcrafted[output_var][input_structure]['train_r2']=train_r2
        performance_handcrafted[output_var][input_structure]['test_r2']=test_r2

-----
HHVEHCNT_mean_norm
BE linear
BE quadratic
NHTS linear
NHTS quadratic
BE and NHTS linear
BE and NHTS quadratic
-----
HHVEHCNT_P_CAP_mean_norm
BE linear
BE quadratic
NHTS linear
NHTS quadratic
BE and NHTS linear
BE and NHTS quadratic
-----
TRPTRANS_1_mean_norm
BE linear
BE quadratic
NHTS linear
NHTS quadratic
BE and NHTS linear
BE and NHTS quadratic
-----
TRPTRANS_2_mean_norm
BE linear
BE quadratic
NHTS linear
NHTS quadratic
BE and NHTS linear
BE and NHTS quadratic
-----
TRPTRANS_3_mean_norm
BE linear
BE quadratic
NHTS linear
NHTS quadratic
BE and NHTS linear
BE and NHTS quadratic


In [9]:
import pickle
with open('outputs/performance_handcrafted.pickle', 'wb') as h:
    pickle.dump(performance_handcrafted, h, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
# get only test r2 for analysis
performance_handcrafted_r2_test = {}
for output_var_key in performance_handcrafted.keys():
    performance_handcrafted_r2_test[output_var_key]={}
    for input_structure_key in performance_handcrafted[output_var_key].keys():
        performance_handcrafted_r2_test[output_var_key][input_structure_key]=\
            performance_handcrafted[output_var_key][input_structure_key]['test_r2']

r2_test_table = pd.DataFrame(performance_handcrafted_r2_test)
r2_test_table

,HHVEHCNT_mean_norm,HHVEHCNT_P_CAP_mean_norm,TRPTRANS_1_mean_norm,TRPTRANS_2_mean_norm,TRPTRANS_3_mean_norm
BE linear,0.084195,0.087349,0.178908,0.222090,0.110670
BE quadratic,0.094053,0.094257,0.223722,0.275076,0.126030
NHTS linear,0.341971,0.337261,0.318877,0.382529,0.169336
NHTS quadratic,0.365544,0.367325,0.317638,0.382133,0.147775
BE and NHTS linear,0.343357,0.338454,0.319964,0.385261,0.173725
BE and NHTS quadratic,0.366108,0.367583,0.323204,0.399684,0.173483


## Combine extracted ResNet layers with NHTS data sets

In [43]:
# Train two other input structures.
# total models: 5 * 2 * 5 * 5 = 250 models.
# This part needs to be refined. Ideally we still need train/val/testing sets. 
method = 'elastic_net'

alpha_list = [10.0, 1.0, 0.1, 0.01, 0.001]
L1_wt_list = [0.01, 0.1, 0.5, 0.9, 0.99]
input_structure_list = ['CNN and NHTS linear', 'CNN BE NHTS linear']

performance_cnn_combined = {}
hyper_param_dic = {}

for output_var in output_var_list:
    print("-----")
    print(output_var)
    performance_cnn_combined[output_var] = {}
    hyper_param_dic[output_var]={}

    for input_structure in input_structure_list:
        print(input_structure)
        
        performance_cnn_combined[output_var][input_structure]={}
        hyper_param_dic[output_var][input_structure]={}
        
        x_train_, x_test_, y_train_, y_test_ = initialize_data_linear_reg(df, BE_var, output_var, input_var, last_layer_dic_train, last_layer_dic_test, size, input_structure)
            
        # search a bit. It takes a while...
        best_train_mse=0.0
        best_test_mse=0.0
        best_train_r2=0.0
        best_test_r2=0.0
        hyper_param_dic[output_var][input_structure]['alpha']=0.0
        hyper_param_dic[output_var][input_structure]['L1_wt']=0.0

        # search 5*5=25 models
        for alpha in alpha_list:
            for L1_wt in L1_wt_list:
                linear_mod_res, train_mse, test_mse, train_r2, test_r2 = Linear_reg_eval(x_train_, x_test_, y_train_, y_test_, method, alpha, L1_wt)
                
                if test_r2 > best_test_r2:
                    best_train_mse=train_mse
                    best_test_mse=test_mse
                    best_train_r2=train_r2
                    best_test_r2=test_r2
                    hyper_param_dic[output_var][input_structure]['alpha']=alpha
                    hyper_param_dic[output_var][input_structure]['L1_wt']=L1_wt
                    
        performance_cnn_combined[output_var][input_structure]['train_mse']=best_train_mse
        performance_cnn_combined[output_var][input_structure]['test_mse']=best_test_mse
        performance_cnn_combined[output_var][input_structure]['train_r2']=best_train_r2
        performance_cnn_combined[output_var][input_structure]['test_r2']=best_test_r2
                

-----
HHVEHCNT_mean_norm
CNN and NHTS linear
CNN BE NHTS linear
-----
HHVEHCNT_P_CAP_mean_norm
CNN and NHTS linear
CNN BE NHTS linear
-----
TRPTRANS_1_mean_norm
CNN and NHTS linear
CNN BE NHTS linear
-----
TRPTRANS_2_mean_norm
CNN and NHTS linear
CNN BE NHTS linear
-----
TRPTRANS_3_mean_norm
CNN and NHTS linear
CNN BE NHTS linear


In [44]:
print(hyper_param_dic)

{'HHVEHCNT_mean_norm': {'CNN and NHTS linear': {'alpha': 0.01, 'L1_wt': 0.9}, 'CNN BE NHTS linear': {'alpha': 0.01, 'L1_wt': 0.9}}, 'HHVEHCNT_P_CAP_mean_norm': {'CNN and NHTS linear': {'alpha': 0.001, 'L1_wt': 0.99}, 'CNN BE NHTS linear': {'alpha': 0.001, 'L1_wt': 0.99}}, 'TRPTRANS_1_mean_norm': {'CNN and NHTS linear': {'alpha': 0.1, 'L1_wt': 0.1}, 'CNN BE NHTS linear': {'alpha': 0.1, 'L1_wt': 0.1}}, 'TRPTRANS_2_mean_norm': {'CNN and NHTS linear': {'alpha': 0.001, 'L1_wt': 0.9}, 'CNN BE NHTS linear': {'alpha': 0.01, 'L1_wt': 0.1}}, 'TRPTRANS_3_mean_norm': {'CNN and NHTS linear': {'alpha': 0.01, 'L1_wt': 0.5}, 'CNN BE NHTS linear': {'alpha': 0.01, 'L1_wt': 0.5}}}


In [47]:
import pickle
with open('outputs/performance_cnn_combined.pickle', 'wb') as h:
    pickle.dump(performance_cnn_combined, h, protocol=pickle.HIGHEST_PROTOCOL)

In [46]:
# get only test r2 for analysis 
performance_cnn_combined_r2_test = {}
for output_var_key in performance_cnn_combined.keys():
    performance_cnn_combined_r2_test[output_var_key]={}
    for input_structure_key in performance_cnn_combined[output_var_key].keys():
        performance_cnn_combined_r2_test[output_var_key][input_structure_key]=\
            performance_cnn_combined[output_var_key][input_structure_key]['test_r2']

r2_test_table = pd.DataFrame(performance_cnn_combined_r2_test)
r2_test_table

,HHVEHCNT_mean_norm,HHVEHCNT_P_CAP_mean_norm,TRPTRANS_1_mean_norm,TRPTRANS_2_mean_norm,TRPTRANS_3_mean_norm
CNN and NHTS linear,0.339674,0.340317,0.336230,0.415407,0.188260
CNN BE NHTS linear,0.339674,0.340322,0.336211,0.415915,0.188249
